# 0 - IMPORTS 

In [241]:
from bs4 import BeautifulSoup
from datetime import datetime

import requests
import pandas as pd
import numpy as np 

# 1 - START 

In [242]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""


In [243]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [244]:
soup 


<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body></html>

In [245]:
soup.body 

<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body>

In [246]:
soup.find_all( 'a', id = 'link2')[0].string

'Lacie'

# 2 - Data Extraction (H&M) - Showcase page data

In [247]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
page = requests.get( url, headers = headers)

soup = BeautifulSoup(page.text, 'html.parser')

products = soup.find('ul', class_ = 'products-listing small')

products_list = products.find_all('article', class_ = 'hm-product-item' )

In [248]:
# Products ID
products_id = [n.get('data-articlecode')for n in products_list]

# Products Category 
products_category = [n.get('data-category')for n in products_list]

# Product name by link class
products_list1 = products.find_all('a', class_ = 'link')

# Products names
products_names = [p.get_text()for p in products_list1]

# Producs Price 
price = products.find_all('span', class_ = 'price regular')
products_price = [p.get_text() for p in price]

In [249]:
data = pd.DataFrame([products_id, products_category, products_names, products_price ]).T
data.columns = ['products_id', 'products_category', 'products_names', 'products_price']

# Scrapy datetime
data['scrapy_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 3 - Pagination at H&M

In [250]:
# Pagination
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
url = "https://www2.hm.com/en_us/men/products/jeans.html"
page = requests.get( url, headers = headers)
soup = BeautifulSoup(page.text, 'html.parser')
total_item = soup.find_all('h2', class_ = 'load-more-heading')[0].get('data-total')
page_number = np.round (int(total_item) / 36)
url02 = url + '?page-size=' + str(int(page_number * 36 ))

In [251]:
# Requests with url02
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
page = requests.get( url02, headers = headers)
soup = BeautifulSoup(page.text, 'html.parser')
products = soup.find('ul', class_ = 'products-listing small')
products_list = products.find_all('article', class_ = 'hm-product-item' )

In [252]:
# Products ID
products_id = [n.get('data-articlecode')for n in products_list]

# Products Category 
products_category = [n.get('data-category')for n in products_list]

# Product name by link class
products_list1 = products.find_all('a', class_ = 'link')

# Products names
products_names = [p.get_text()for p in products_list1]

# Producs Price 
price = products.find_all('span', class_ = 'price regular')
products_price = [p.get_text() for p in price]

In [253]:
data = pd.DataFrame([products_id, products_category, products_names, products_price ]).T
data.columns = ['products_id', 'products_category', 'products_names', 'products_price']

# Scrapy datetime
data['scrapy_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 4 - Products colors

## 4.1 Just one product

In [254]:
# API Requests
url = 'https://www2.hm.com/en_us/productpage.1130309007.html'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.text, 'html.parser')

# Product color
product_list = soup.find_all('a', class_ ="filter-option miniature")
color_name = [p.get('data-color')for p in product_list]
products_id = [p.get('data-articlecode') for p in product_list]

#Data frame with color names and products id 
df_color = pd.DataFrame([products_id, color_name]).T
df_color.columns = ['products_id', 'color_name']

# style ID + color id
df_color['style_id'] = df_color['products_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['products_id'].apply(lambda x: x[-3:])

## 4.2 Multiple Products 

In [255]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

df_detalis = pd.DataFrame()

for i in range (len (data)):

    url = 'https://www2.hm.com/en_us/productpage.'+ data.loc[i, 'products_id'] + '.html'
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Product color
    product_list = soup.find_all('a', class_ ="filter-option miniature")
    color_name = [p.get('data-color')for p in product_list]
    products_id = [p.get('data-articlecode') for p in product_list]

    #Data frame with color names and products id 
    df_color = pd.DataFrame([products_id, color_name]).T
    df_color.columns = ['products_id', 'color_name']

    # style ID + color id
    df_color['style_id'] = df_color['products_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['products_id'].apply(lambda x: x[-3:])

    df_detalis = pd.concat([df_detalis, df_color], axis= 0 )

In [256]:
# Style_id and color_id for data
data['style_id'] = data['products_id'].apply(lambda x: x[:-3])
data['color_id'] = data['products_id'].apply(lambda x: x[-3:])

# 5 - End of web scraping - H&M Showroom of masculinity jeans

In [257]:
# cleaning the data 
df_detalis2 = df_detalis.drop_duplicates()

df_raw = pd.merge(data, df_detalis2[['products_id', 'color_name']], how = 'left', on = 'products_id')
df_raw2 = df_raw.drop_duplicates()

In [260]:
df_raw2

,products_id,products_category,products_names,products_price,scrapy_time,style_id,color_id,color_name
0,1130309007,men_jeans_loose,Baggy Jeans,$ 39.99,2024-03-11 18:22:06,1130309,007,Dark denim gray
1,0979945038,men_jeans_loose,Loose Jeans,$ 39.99,2024-03-11 18:22:06,0979945,038,Denim gray
2,1130309019,men_jeans_loose,Baggy Jeans,$ 39.99,2024-03-11 18:22:06,1130309,019,Light denim blue
3,1130274012,men_jeans_loose,Loose Jeans,$ 39.99,2024-03-11 18:22:06,1130274,012,Denim black
4,1130274013,men_jeans_loose,Loose Jeans,$ 39.99,2024-03-11 18:22:06,1130274,013,Light denim blue
...,...,...,...,...,...,...,...,...
101,0875105017,men_jeans_relaxed,Relaxed Jeans,$ 39.99,2024-03-11 18:22:06,0875105,017,Dark denim blue
102,1210576004,men_jeans_regular,Straight Regular Jeans,$ 39.99,2024-03-11 18:22:06,1210576,004,NaN
103,0811993068,men_jeans_regular,Straight Regular Jeans,$ 39.99,2024-03-11 18:22:06,0811993,068,Dark denim blue
104,0985159008,men_jeans_skinny,Skinny Jeans,$ 29.99,2024-03-11 18:22:06,0985159,008,Dark gray
